In [6]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.api import VAR

In [7]:
df = pd.read_csv('final_daily.csv')
df['date']=pd.to_datetime(df['date'])
df=df[df['date']<datetime.datetime(2023,1,1)]

In [8]:
df1=df[['date','price','demand']]
#df2=df[['date','demand','pric']]

In [9]:
price_train=df1[df1['date']<datetime.datetime(2022,1,1)]
price_test=df1[df1['date']>=datetime.datetime(2022,1,1)]
#demand_train=df2[df2['date']<datetime.datetime(2022,1,1)]
#demand_test=df2[df2['date']>=datetime.datetime(2022,1,1)]
#weather_train=df3[df3['date']<datetime.datetime(2022,1,1)]
#weather_test=df3[df3['date']>=datetime.datetime(2022,1,1)]

In [16]:
price_train.type()

AttributeError: 'DataFrame' object has no attribute 'type'

In [17]:
model = VAR(price_train[['price','demand']])

In [18]:
res = model.fit()

In [24]:
res.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 27, Feb, 2023
Time:                     14:44:56
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                    24.3878
Nobs:                     730.000    HQIC:                   24.3646
Log likelihood:          -10953.4    FPE:                3.75903e+10
AIC:                      24.3500    Det(Omega_mle):     3.72833e+10
--------------------------------------------------------------------
Results for equation price
               coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------
const            -6.330429         3.720653           -1.701           0.089
L1.price          0.647419         0.029885           21.664           0.000
L1.demand         0.000037         0.000011            3.379           0.001

In [26]:
nobs =1 
# Get the lag order
lag_order = res.k_ar
print(lag_order)  #> 4

# Input data for forecasting
forecast_input = price_train[['price','demand']].values[-lag_order:]
forecast_input

1


array([[3.111375e+01, 3.624920e+05]])

In [28]:
fc = res.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=price_train[['price','demand']].index[-nobs:], columns=price_train[['price','demand']].columns + '_2d')
df_forecast

,price_2d,demand_2d
730,27.067863,362332.772402


In [19]:
res.predict(steps=1)

TypeError: forecast() missing 1 required positional argument: 'y'

In [10]:
price_train2 = price_train.drop(['date'], axis = 1)

In [12]:
fc = pd.DataFrame()

for i in range(-365, 0):
    model = VAR(df[['price','demand']].iloc[:len(df) + i])
    res = model.fit()
    forecast = res.forecast('price',steps=1)
    fc = pd.concat([fc, forecast])

TypeError: Cannot cast array data from dtype('float64') to dtype('<U32') according to the rule 'safe'

In [ ]:
res.forecast()

In [ ]:
plt.plot(price_test['price'].reset_index(drop=True), label='Truth')
plt.plot(fc.values[:, 0], label='Prediction')
plt.legend()

In [ ]:
np.square(fc.values.reshape(-1, 1) - demand_test['demand'].values.reshape(-1, 1)).mean()

In [ ]:
np.abs(fc.values.reshape(-1, 1) - demand_test['demand'].values.reshape(-1, 1)).mean()

In [ ]:
def mape(x, y):
  error = abs(x-y)/abs(x)
  error[error > 1] = 1
  return np.mean(error)

mape(fc.values.reshape(-1, 1), demand_test['demand'].values.reshape(-1, 1))